<a href="https://colab.research.google.com/github/nickchak21/QuarkGluonClassifiers/blob/master/PFN_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install energyflow
!pip install h5py

     |████████████████████████████████| 686kB 2.7MB/s 
     |████████████████████████████████| 2.9MB 38.4MB/s 
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0


In [2]:
!python --version

Python 3.6.8


In [3]:
!pip install POT

     |████████████████████████████████| 307kB 2.7MB/s 


In [4]:
!python -c "import energyflow; energyflow.utils.get_examples()"


Summary of examples:
cnn_example.py exists at /root/.energyflow/examples
dnn_example.py exists at /root/.energyflow/examples
efp_example.py exists at /root/.energyflow/examples
efn_example.py exists at /root/.energyflow/examples
pfn_example.py exists at /root/.energyflow/examples



In [0]:
%pycat /root/.energyflow/examples/pfn_example.py

In [0]:
rm /root/.energyflow/examples/pfn_example.py

In [28]:
%%writefile /root/.energyflow/examples/pfn_example.py
"""An example involving Particle Flow Networks (PFNs), which were 
introduced in [1810.05165](https://arxiv.org/abs/1810.05165). The 
[`PFN`](../docs/archs/#pfn) class is used to construct the 
network architecture. The output of the example is a plot of the 
ROC curves obtained by the PFN as well as the jet mass and 
constituent multiplicity observables.
"""

# standard library imports
from __future__ import absolute_import, division, print_function

# standard numerical library imports
import numpy as np

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, remap_pids, to_categorical

# attempt to import sklearn
try:
    from sklearn.metrics import roc_auc_score, roc_curve
except:
    print('please install scikit-learn in order to make ROC curves')
    roc_curve = False

# attempt to import matplotlib
try:
    import matplotlib.pyplot as plt
except:
    print('please install matploltib in order to make plots')
    plt = False

################################### SETTINGS ###################################
# the commented values correspond to those in 1810.05165
###############################################################################

# data controls, can go up to 2000000 for full dataset
train, val, test = 675000, 90000, 135000
# train, val, test = 1000000, 200000, 200000
use_pids = True

# network architecture parameters
Phi_sizes, F_sizes = (100, 100, 128), (100, 100, 100)
# Phi_sizes, F_sizes = (100, 100, 256), (100, 100, 100)

# network training parameters
num_epoch = 20
batch_size = 500

################################################################################

# load data
X, y = qg_jets.load(train + val + test)

# convert labels to categorical
Y = to_categorical(y, num_classes=2)

print('Loaded quark and gluon jets')

# preprocess by centering jets and normalizing pts
for x in X:
    mask = x[:,0] > 0
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg
    x[mask,0] /= x[:,0].sum()

# handle particle id channel
if use_pids:
    remap_pids(X, pid_i=3)
else:
    X = X[:,:,:3]

print('Finished preprocessing')

# do train/val/test split 
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=val, test=test)

print('Done train/val/test split')
print('Model summary:')

# build architecture
pfn = PFN(input_dim=X.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes)

# train model
pfn.fit(X_train, Y_train,
          epochs=num_epoch,
          batch_size=batch_size,
          validation_data=(X_val, Y_val),
          verbose=1)

# get predictions on test data
preds = pfn.predict(X_test, batch_size=1000)

# get ROC curve if we have sklearn
if roc_curve:
    pfn_fp, pfn_tp, threshs = roc_curve(Y_test[:,1], preds[:,1])

    # get area under the ROC curve
    auc = roc_auc_score(Y_test[:,1], preds[:,1])
    print()
    print('PFN AUC:', auc)
    print()

    # make ROC curve plot if we have matplotlib
    if plt:

        # get multiplicity and mass for comparison
        masses = np.asarray([ef.ms_from_p4s(ef.p4s_from_ptyphims(x).sum(axis=0)) for x in X])
        mults = np.asarray([np.count_nonzero(x[:,0]) for x in X])
        mass_fp, mass_tp, threshs = roc_curve(Y[:,1], -masses)
        mult_fp, mult_tp, threshs = roc_curve(Y[:,1], -mults)

        # some nicer plot settings 
        plt.rcParams['figure.figsize'] = (4,4)
        plt.rcParams['font.family'] = 'serif'
        plt.rcParams['figure.autolayout'] = True

        # plot the ROC curves
        plt.plot(pfn_tp, 1-pfn_fp, '-', color='black', label='PFN')
        plt.plot(mass_tp, 1-mass_fp, '-', color='blue', label='Jet Mass')
        plt.plot(mult_tp, 1-mult_fp, '-', color='red', label='Multiplicity')

        # axes labels
        plt.xlabel('Quark Jet Efficiency')
        plt.ylabel('Gluon Jet Rejection')

        # axes limits
        plt.xlim(0, 1)
        plt.ylim(0, 1)

        # make legend and show plot
        plt.legend(loc='lower left', frameon=False)
        plt.show()

Writing /root/.energyflow/examples/pfn_example.py


In [29]:
!python /root/.energyflow/examples/pfn_example.py

Using TensorFlow backend.
tcmalloc: large alloc 4089602048 bytes == 0x1d0884000 @  0x7f20deb7c1e7 0x7f20dc6dbf71 0x7f20dc73f55d 0x7f20dc73f733 0x7f20dc7dd768 0x7f20dc7ddfc4 0x7f20dc7de112 0x5673a3 0x5a04ce 0x7f20dc72b06d 0x50a84f 0x50c549 0x5081d5 0x50a020 0x50aa1d 0x50c549 0x5081d5 0x5895e1 0x5a04ce 0x7f20dc72b06d 0x50a84f 0x50c549 0x5081d5 0x50a020 0x50aa1d 0x50c549 0x5081d5 0x50a020 0x50aa1d 0x50c549 0x5081d5
Loaded quark and gluon jets
tcmalloc: large alloc 1233903616 bytes == 0xf110c000 @  0x7f20deb7d615 0x579495 0x50d3b0 0x5081d5 0x50a020 0x50aa1d 0x50c549 0x5081d5 0x50a020 0x50aa1d 0x50d320 0x5081d5 0x50b3a3 0x635082 0x635137 0x6388ef 0x639491 0x4b0f60 0x7f20de779b97 0x5b2eaa
Finished preprocessing
tcmalloc: large alloc 3067207680 bytes == 0xf110c000 @  0x7f20deb7c1e7 0x7f20dc6dbf71 0x7f20dc73f55d 0x7f20dc73f733 0x7f20dc7caa2a 0x7f20dc7cae78 0x50c81e 0x5081d5 0x50a020 0x50aa1d 0x50c549 0x5081d5 0x50a020 0x50aa1d 0x50d320 0x5081d5 0x50b3a3 0x635082 0x635137 0x6388ef 0x639491 0x4b